# KNN

First run the file [/notebooks/CaseStudy1/Clean%20Data%20(From%20Book).ipynb](/notebooks/CaseStudy1/Clean%20Data%20(From%20Book).ipynb)


In [21]:
OFFLINE_SOURCE = '../data/offline.final.trace.csv'

NUM_ROWS_FOR_SAMPLE = 1000 # out of 914,951 rows

In [3]:
offline = read.csv(OFFLINE_SOURCE, row.names=1)

In [22]:
trainSample = offline[sample(nrow(offline), NUM_ROWS_FOR_SAMPLE), ]

In [9]:
#Create a special factor that contains all of the unique combinations 
#of the observed (x, y) pairs for the 166 locations. 

offline$posXY = paste(offline$posX, offline$posY, sep = "-")

#create a list of data frames for every combination of (x, y), angle, and access point
byLocAngleAP = with(offline,
by(offline, list(posXY, angle, mac),
function(x) x))
   
#create summary statistics on each of the data frames
   
signalSummary =
    lapply(byLocAngleAP,
    function(oneLoc) {
    ans = oneLoc[1, ]
    ans$medSignal = median(oneLoc$signal)
    ans$avgSignal = mean(oneLoc$signal)
    ans$num = length(oneLoc$signal)
    ans$sdSignal = sd(oneLoc$signal)
    ans$iqrSignal = IQR(oneLoc$signal)
    ans
    })
   


In [10]:
offlineSummary = do.call("rbind", signalSummary)

In [61]:
# provide a scalar x and y along with a dataframe that has the columns posX and posY
# will return the dataframe sorted by whose posX/posY values are closest to the x/y provided
findNN.with.x.y = function(x, y, trainSubset) {
  diffs = apply(trainSubset[c('posX', 'posY')], 1, function(row) row - c(x, y))
  dists = apply(diffs, 2, function(x) sqrt(sum(x^2)) )
  closest = order(dists)
  return(trainSubset[closest,])
}

In [62]:
# Idealy, this would have a bunch of posX=4, posY=8 rows
head(findNN.with.x.y(4, 8, trainSample))

,time,posX,posY,orientation,mac,signal,rawTime,angle,posXY
625381,2006-02-11 14:55:20,4,8,45.2,00:0f:a3:39:dd:cd,-72,1.139691e+12,45,4-8
625644,2006-02-11 14:55:30,4,8,45.2,00:14:bf:b1:97:8a,-52,1.139691e+12,45,4-8
637234,2006-02-11 15:04:45,3,8,225.8,00:14:bf:b1:97:8d,-60,1.139692e+12,225,3-8
585872,2006-02-11 14:01:06,4,7,271.0,00:0f:a3:39:dd:cd,-67,1.139688e+12,270,4-7
580932,2006-02-11 13:56:48,4,7,45.9,00:0f:a3:39:dd:cd,-61,1.139688e+12,45,4-7
638305,2006-02-11 15:05:55,3,8,315.6,00:14:bf:3b:c7:c6,-67,1.139692e+12,315,3-8
